In [1]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from itertools import product
from base import *
from exputils import *

# Load data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['change_1', 'change_2', 'change_3']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

train_dict = {p: read_train(p, dropbox_dir) for p in pairs}
ohlc_featured_train = load_model('ohlc_featured_train.pkl')

In [3]:
model = load_model('models/lin_regression.pkl')

# Setup

In [4]:
features = ['support', 'change_1', 'change_2', 'change_3']

window_sizes = [4]
shifts = [1, 2, 3]

DAY = 48
max_hold_time = 5 * DAY
stop_loss = 0.05
fee = 0.0025

min_profit_targets = [round(i / 100, 2) for i in range(0, 6)]

args = product(pairs, window_sizes, min_profit_targets)
args = list(args)
len(args)

114

# Train trading stats

In [5]:
def trading_stats(data, pair, willing_loss, min_profit_target):
    contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
    dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
    wallet = {'btc': 100}
    exchange = BackTestExchange(dm, wallet, 0.0025)
    strategy = BaseMLStrategy(exchange, 
                            dm, 
                            contract_pair, 
                            willing_loss=willing_loss,  
                            min_profit_target=min_profit_target,
                            model=model)
    strategy.trade()
    strategy_stats = strategy.stats(contract_pair['priceContract']).report()
    return strategy_stats


def trading_stats_f(args):
    pair, window_size, min_profit_target = args
    
    data = train_dict[pair]
    data[features] = ohlc_featured_train[pair][window_size]
    stats = trading_stats(data, pair, stop_loss, min_profit_target)
    return pair, window_size, stop_loss, min_profit_target, stats

In [6]:
results = run_parallel(trading_stats_f, args)

param_cols = ['pair', 'window_size', 'stop_loss', 'min_profit_target']
stats_cols = ['startAmount', 'numberOfTrades', 'totalProfit', 'avgProfit',
    'winPercentage', 'avgWinTrade', 'avgLossTrade', 'profitFactor',
    'maxDrawdown', 'avgTimeToClose', 'minTimeToClose', 'maxTimeToClose']

result_df = pd.DataFrame(results, columns=param_cols+['stats'])

for c in stats_cols:
    result_df[c] = result_df['stats'].apply(lambda x: x.iloc[:, 0][c])
    
result_df = result_df.drop('stats', axis=1)
result_df.to_pickle('train_results.pkl')

In [7]:
result_df

,pair,window_size,stop_loss,min_profit_target,startAmount,numberOfTrades,totalProfit,avgProfit,winPercentage,avgWinTrade,avgLossTrade,profitFactor,maxDrawdown,avgTimeToClose,minTimeToClose,maxTimeToClose
0,BTC_ETH,4,0.0500,0.0400,100.0000,134.0000,236.2758,1.7633,59.7015,5.9982,-4.5107,1.9700,20.9432,14.1455,0.2500,186.5000
1,BTC_XRP,4,0.0500,0.0300,100.0000,59.0000,19.8805,0.3370,54.2373,4.4272,-4.5107,1.1632,24.9437,36.6186,0.2500,408.2500
2,BTC_ETH,4,0.0500,0.0300,100.0000,195.0000,186.6605,0.9572,56.4103,5.1588,-4.4800,1.4902,26.3981,16.9038,0.2500,284.2500
3,BTC_ETH,4,0.0500,0.0500,100.0000,75.0000,258.7193,3.4496,66.6667,7.4297,-4.5107,3.2943,12.6093,16.0500,0.2500,186.5000
4,BTC_XRP,4,0.0500,0.0200,100.0000,112.0000,69.5624,0.6211,66.9643,3.1528,-4.5107,1.4168,12.5421,37.0915,0.2500,418.0000
5,BTC_ETH,4,0.0500,0.0200,100.0000,276.0000,127.4884,0.4619,57.2464,4.1591,-4.4886,1.2407,31.1292,16.0707,0.2500,284.2500
6,BTC_XRP,4,0.0500,0.0100,100.0000,229.0000,108.4945,0.4738,76.4192,2.0118,-4.5107,1.4454,14.5081,25.2194,0.2500,418.0000
7,BTC_ETH,4,0.0500,0.0100,100.0000,393.0000,121.2117,0.3084,64.1221,2.9945,-4.4922,1.1914,34.5046,14.8715,0.2500,284.2500
8,BTC_XRP,4,0.0500,0.0000,100.0000,861.0000,72.5248,0.0842,89.5470,0.6206,-4.5107,1.1786,27.6946,9.1220,0.2500,418.0000
9,BTC_ETH,4,0.0500,0.0000,100.0000,672.0000,66.7946,0.0994,75.2976,1.6067,-4.4950,1.0895,38.9960,11.2121,0.2500,284.2500


# Tmp

In [ ]:
from modeling.strategy import Strategy


In [ ]:
def trading_stats2(data, pair, willing_loss, min_profit_target):
    contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
    dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
    wallet = {'btc': 100}
    exchange = BackTestExchange(dm, wallet, 0.0025)
    strategy = BaseMLStrategy(exchange, 
                            dm, 
                            contract_pair, 
                            willing_loss=willing_loss,  
                            min_profit_target=min_profit_target,
                            model=model)
    strategy.trade()
    strategy_stats = strategy.stats(contract_pair['priceContract'])
    return strategy_stats

In [ ]:
pair = 'BTC_ETH'
data = train_dict[pair]
data[features] = ohlc_featured_train[pair][4]
min_profit_target = 0.03

In [ ]:
stats.report()